# Statistical models in R
This notebook is covering:
1. ARIMA-models.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [13]:
%%R

hub1_name <- "the"
validation_size <- 250
test_size <- 250
window_size <- 5




In [14]:
%%R
source("arima.r")

output <- arima_system(hub1_name, validation_size, test_size, window_size, verbose = TRUE, save = TRUE)

[1] "Order: 1" "Order: 1" "Order: 4"
[1] "the: Mean Absolute Error: 2.07516663420426"
[1] "the: Root Mean Squared Error: 2.74040016558413"


In [15]:
%%R
hub_predictions <- output$predictions$hub
hub_actuals <- output$actuals$hub
hub_forecast_comparison <- data.frame(
  Actual = hub_actuals,
  Forecasted = hub_predictions
)

In [16]:
%R -o hub_forecast_comparison
%R -o hub1_name

hub_forecast_comparison = hub_forecast_comparison

actual_trace = go.Scatter(
    x=hub_forecast_comparison.index,
    y=hub_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub_forecast_comparison.index,
    y=hub_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title=hub1_name.item()+ ': ARIMA Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()
